In [21]:
from Bio import SeqIO

In [41]:
def splitPeptide(peptide, maxed):
    length = len(peptide)
    splits = []
    
    for i in range(0, length):
        character = peptide[i]
        
        toAdd=""
        toAdd+=character
        splits.append(toAdd)
        for j in range(i+1, length):
            toAdd+=peptide[j]
            if(maxSize(toAdd, maxed)):
                splits.append(toAdd)
    return splits
            
            
    

In [94]:
def splitDictPeptide(peptide, maxed):
    length = len(peptide)
    splits = []
    splitRef = []
    for i in range(0, length):
        character = peptide[i]
        
        toAdd=""
        toAdd+=character
        splits.append(toAdd)
        ref = []
        ref.append(i)
        temp = list(ref)
        splitRef.append(temp)
        
        for j in range(i+1, length):
            toAdd+=peptide[j]
            if(maxSize(toAdd, maxed)):
                
                splits.append(toAdd)
                ref.append(j)
                temp = list(ref)
                
                splitRef.append(temp)
                temp=[]
      
        print(splitRef)
        ref = []
        splitRef = []
        
    return splits

In [78]:
def combinePeptide(splits, mined, maxed):
    combine = []
    for i in range(0, len(splits)):
        if(minSize(splits[i], mined)):
            #Add linear peptide, can include option if necessary
            combine.append(splits[i])
        toAdd=""
        for j in range(i+1, len(splits)):
           
            toAdd+=splits[i]
            toAdd+=splits[j]
            if(maxSize(toAdd, maxed) and minSize(toAdd, mined)):
                
                combine.append(toAdd)
            
            toAdd = ""
            toAdd+=splits[j]
            toAdd+=splits[i]
            if(maxSize(toAdd, maxed) and minSize(toAdd, mined)):
                
                combine.append(toAdd)
        
            toAdd=""
    return combine
     
    

In [79]:
def maxSize(split, maxed):
    if(len(split)>=maxed):
        return False
    return True

In [80]:
def minSize(split, mined):
    if(len(split)<mined):
        return False
    return True

In [81]:
def addSequenceList(input_file):
    
    fasta_sequences = SeqIO.parse(open(input_file),'fasta')
    sequenceDictionary = {}
    for fasta in fasta_sequences:
        name, sequence = fasta.id, fasta.seq.tostring()
        sequenceDictionary[name] = sequence
    return sequenceDictionary

In [95]:
maxed = 12
mined=0
splits = splitDictPeptide("AKLjf", maxed)


[[0], [0, 1], [0, 1, 2], [0, 1, 2, 3], [0, 1, 2, 3, 4]]
[[1], [1, 2], [1, 2, 3], [1, 2, 3, 4]]
[[2], [2, 3], [2, 3, 4]]
[[3], [3, 4]]
[[4]]


In [83]:
print(splits)
print(len(splits))

['A', 'AK', 'AKL', 'AKLj', 'AKLjf', 'K', 'KL', 'KLj', 'KLjf', 'L', 'Lj', 'Ljf', 'j', 'jf', 'f']
15


In [66]:
combine = combinePeptide(splits, mined, maxed)
print(combine)
print(len(combine))

['A', 'AAK', 'AKA', 'AAKL', 'AKLA', 'AAKLj', 'AKLjA', 'AAKLjf', 'AKLjfA', 'AK', 'KA', 'AKL', 'KLA', 'AKLj', 'KLjA', 'AKLjf', 'KLjfA', 'AL', 'LA', 'ALj', 'LjA', 'ALjf', 'LjfA', 'Aj', 'jA', 'Ajf', 'jfA', 'Af', 'fA', 'AK', 'AKAKL', 'AKLAK', 'AKAKLj', 'AKLjAK', 'AKAKLjf', 'AKLjfAK', 'AKK', 'KAK', 'AKKL', 'KLAK', 'AKKLj', 'KLjAK', 'AKKLjf', 'KLjfAK', 'AKL', 'LAK', 'AKLj', 'LjAK', 'AKLjf', 'LjfAK', 'AKj', 'jAK', 'AKjf', 'jfAK', 'AKf', 'fAK', 'AKL', 'AKLAKLj', 'AKLjAKL', 'AKLAKLjf', 'AKLjfAKL', 'AKLK', 'KAKL', 'AKLKL', 'KLAKL', 'AKLKLj', 'KLjAKL', 'AKLKLjf', 'KLjfAKL', 'AKLL', 'LAKL', 'AKLLj', 'LjAKL', 'AKLLjf', 'LjfAKL', 'AKLj', 'jAKL', 'AKLjf', 'jfAKL', 'AKLf', 'fAKL', 'AKLj', 'AKLjAKLjf', 'AKLjfAKLj', 'AKLjK', 'KAKLj', 'AKLjKL', 'KLAKLj', 'AKLjKLj', 'KLjAKLj', 'AKLjKLjf', 'KLjfAKLj', 'AKLjL', 'LAKLj', 'AKLjLj', 'LjAKLj', 'AKLjLjf', 'LjfAKLj', 'AKLjj', 'jAKLj', 'AKLjjf', 'jfAKLj', 'AKLjf', 'fAKLj', 'AKLjf', 'AKLjfK', 'KAKLjf', 'AKLjfKL', 'KLAKLjf', 'AKLjfKLj', 'KLjAKLjf', 'AKLjfKLjf', 'KLjf

In [40]:
sequenceDictionary = addSequenceList("Example.fasta")
for key, value in sequenceDictionary.items():
    
    splits = splitPeptide(value, size)
    combine = combinePeptide(splits, size)
    print(len(splits))
    print(len(combine))
    
    break;

C:\Users\Arpit\Anaconda3\lib\site-packages\Bio\Seq.py:354: BiopythonDeprecationWarning: This method is obsolete; please use str(my_seq) instead of my_seq.tostring().
  BiopythonDeprecationWarning)


2662
3916942
